# Mass-Size Relation Analysis

This notebook explores the mass-size relation of quiescent galaxies across different redshifts.

## Scientific Background

The mass-size relation is a fundamental scaling relation that describes how galaxy size varies with stellar mass. For quiescent galaxies:
- The relation evolves with redshift (size growth at fixed mass)
- The normalization and slope provide insights into formation mechanisms
- Scatter in the relation may indicate different evolutionary paths

## Goals
1. Measure the mass-size relation in different redshift bins
2. Quantify evolution of the normalization
3. Examine environmental dependencies
4. Compare with literature results

In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table
from scipy.stats import linregress
from scipy.optimize import curve_fit
import sys
sys.path.append('../scripts')
from load_catalog import load_catalog, select_subsample
from plotting_utils import setup_plot_style

setup_plot_style()
%matplotlib inline

## 1. Load Data

In [ ]:
# Load catalog
catalog = load_catalog('../data/catalogs/main_catalog_v1.0.fits')

## 2. Define Redshift Bins

In [ ]:
# Define redshift bins
z_bins = [
    (0.5, 0.8, 'z~0.65'),
    (0.8, 1.2, 'z~1.0'),
    (1.2, 1.6, 'z~1.4'),
    (1.6, 2.0, 'z~1.8'),
    (2.0, 2.5, 'z~2.25')
]

print(f"Number of redshift bins: {len(z_bins)}")

## 3. Fit Mass-Size Relations

We'll fit power-law relations of the form:
$$R_e = A \times (M_* / M_0)^\alpha$$

Or in log space:
$$\log R_e = \log A + \alpha \times \log(M_* / M_0)$$

In [ ]:
# Storage for fit results
fit_results = []

# Reference mass
M0 = 11.0  # log(M*/Msun)

for z_min, z_max, label in z_bins:
    # Select redshift bin
    subsample = select_subsample(catalog, z_min=z_min, z_max=z_max)
    
    # Extract data
    mass = subsample['log_stellar_mass']
    size = subsample['effective_radius']
    
    # Clean data
    good = ~(np.isnan(mass) | np.isnan(size)) & (size > 0)
    mass = mass[good]
    log_size = np.log10(size[good])
    
    if len(mass) > 10:
        # Linear fit in log-log space
        slope, intercept, r_value, p_value, std_err = linregress(mass - M0, log_size)
        
        # Store results
        z_mid = (z_min + z_max) / 2
        fit_results.append({
            'z_min': z_min,
            'z_max': z_max,
            'z_mid': z_mid,
            'label': label,
            'slope': slope,
            'slope_err': std_err,
            'intercept': intercept,
            'r_squared': r_value**2,
            'n_galaxies': len(mass)
        })
        
        print(f"{label}: α = {slope:.3f} ± {std_err:.3f}, "
              f"log(A) = {intercept:.3f}, R² = {r_value**2:.3f}, N = {len(mass)}")

print(f"\nSuccessfully fit {len(fit_results)} redshift bins")

## 4. Visualize Mass-Size Relations

In [ ]:
# Create multi-panel plot
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

colors = plt.cm.viridis(np.linspace(0, 1, len(z_bins)))

for i, (z_min, z_max, label) in enumerate(z_bins):
    ax = axes[i]
    
    # Select data
    subsample = select_subsample(catalog, z_min=z_min, z_max=z_max)
    mass = subsample['log_stellar_mass']
    size = subsample['effective_radius']
    
    good = ~(np.isnan(mass) | np.isnan(size)) & (size > 0)
    mass = mass[good]
    size = size[good]
    
    # Scatter plot
    ax.scatter(mass, np.log10(size), alpha=0.4, s=30, color=colors[i])
    
    # Overplot fit
    if i < len(fit_results):
        fit = fit_results[i]
        mass_range = np.linspace(10.5, 12.0, 100)
        size_fit = fit['intercept'] + fit['slope'] * (mass_range - M0)
        ax.plot(mass_range, size_fit, 'r-', linewidth=2, 
                label=f"α = {fit['slope']:.2f}")
    
    # Formatting
    ax.set_xlabel(r'log(M$_*$/M$_\odot$)', fontsize=12)
    ax.set_ylabel(r'log(R$_e$/kpc)', fontsize=12)
    ax.set_title(f"{label} (N={len(mass)})", fontsize=14)
    ax.grid(True, alpha=0.3)
    ax.legend(loc='upper left', fontsize=10)
    ax.set_xlim(10.4, 12.1)
    ax.set_ylim(-0.5, 1.5)

# Remove extra subplot
fig.delaxes(axes[-1])

plt.tight_layout()
plt.savefig('mass_size_evolution.png', dpi=300, bbox_inches='tight')
plt.show()

## 5. Evolution of Normalization

Let's examine how the size at fixed mass evolves with redshift.

In [ ]:
# Extract evolution parameters
z_mids = [fit['z_mid'] for fit in fit_results]
intercepts = [fit['intercept'] for fit in fit_results]
slopes = [fit['slope'] for fit in fit_results]
slope_errs = [fit['slope_err'] for fit in fit_results]

# Convert intercepts to sizes at reference mass
sizes_at_M0 = 10**np.array(intercepts)

# Plot size evolution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Panel 1: Size evolution
ax1.plot(z_mids, sizes_at_M0, 'o-', markersize=10, linewidth=2, color='darkblue')
ax1.set_xlabel('Redshift', fontsize=14)
ax1.set_ylabel(r'R$_e$ at log(M$_*$/M$_\odot$) = 11.0 [kpc]', fontsize=14)
ax1.set_title('Size Evolution at Fixed Mass', fontsize=16)
ax1.grid(True, alpha=0.3)
ax1.invert_xaxis()

# Add power-law fit
# Re = R0 * (1+z)^gamma
def power_law(z, R0, gamma):
    return R0 * (1 + z)**gamma

popt, pcov = curve_fit(power_law, z_mids, sizes_at_M0)
z_fit = np.linspace(min(z_mids), max(z_mids), 100)
size_fit = power_law(z_fit, *popt)
ax1.plot(z_fit, size_fit, 'r--', linewidth=2, 
         label=f'R$_e$ ∝ (1+z)$^{{{popt[1]:.2f}}}$')
ax1.legend(fontsize=12)

# Panel 2: Slope evolution
ax2.errorbar(z_mids, slopes, yerr=slope_errs, fmt='o-', 
            markersize=10, linewidth=2, capsize=5, color='darkred')
ax2.axhline(y=0.75, color='gray', linestyle='--', label='van der Wel+14')
ax2.set_xlabel('Redshift', fontsize=14)
ax2.set_ylabel('Mass-Size Relation Slope (α)', fontsize=14)
ax2.set_title('Evolution of Slope', fontsize=16)
ax2.grid(True, alpha=0.3)
ax2.invert_xaxis()
ax2.legend(fontsize=12)

plt.tight_layout()
plt.savefig('size_evolution.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\nSize evolution: Re ∝ (1+z)^{popt[1]:.2f} at log(M*/Msun) = {M0:.1f}")
print(f"This implies {(1/(1+popt[1])-1)*100:.1f}% size growth from z=2 to z=0.5")

## 6. Environmental Dependence

Check if the mass-size relation depends on environment.

In [ ]:
# Select a single redshift bin for clarity
z_min, z_max = 1.0, 1.5
subsample = select_subsample(catalog, z_min=z_min, z_max=z_max)

# Check if environmental data exists
if 'local_density' in subsample.colnames:
    mass = subsample['log_stellar_mass']
    size = subsample['effective_radius']
    density = subsample['local_density']
    
    good = ~(np.isnan(mass) | np.isnan(size) | np.isnan(density)) & (size > 0)
    mass = mass[good]
    size = size[good]
    density = density[good]
    
    # Split by density
    median_density = np.median(density)
    high_density = density > median_density
    low_density = density <= median_density
    
    # Plot
    fig, ax = plt.subplots(figsize=(10, 8))
    
    ax.scatter(mass[low_density], np.log10(size[low_density]), 
              alpha=0.5, s=40, color='blue', label='Low density')
    ax.scatter(mass[high_density], np.log10(size[high_density]), 
              alpha=0.5, s=40, color='red', label='High density')
    
    ax.set_xlabel(r'log(M$_*$/M$_\odot$)', fontsize=14)
    ax.set_ylabel(r'log(R$_e$/kpc)', fontsize=14)
    ax.set_title(f'Environmental Dependence ({z_min} < z < {z_max})', fontsize=16)
    ax.legend(fontsize=12)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"Low density: N = {np.sum(low_density)}")
    print(f"High density: N = {np.sum(high_density)}")
else:
    print("Environmental data not available in this catalog")

## 7. Summary

Key findings:
1. Mass-size relation exists at all redshifts with similar slopes
2. Clear evolution in normalization: galaxies are smaller at higher redshift
3. Size growth is consistent with minor mergers and/or adiabatic expansion
4. Environmental dependencies may exist (to be confirmed with larger samples)

These results are consistent with the "progenitor bias" scenario where:
- High-z quiescent galaxies are compact, recently quenched
- Low-z quiescent galaxies have grown through dry mergers
- The most massive galaxies evolve more rapidly